In [1]:
# Importing all needed functions
import os
import io
from contextlib import redirect_stdout
from astropy.io import fits
import glob
from Calculating_det_angles import estimate_source_angles_detectors  # Importing ma'am's function
from Tools import tools
import threading
from queue import Queue
import queue
import traceback
import pandas as pd
import os
from collections import Counter
import sys

import pandas as pd



In [2]:
data_set_path = r"D:\GRB_data\500_data_set"
df = pd.read_csv('all_events.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6372 entries, 0 to 6371
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   event_name   6372 non-null   object 
 1   event_type   6372 non-null   object 
 2   reliability  6372 non-null   float64
dtypes: float64(1), object(2)
memory usage: 149.5+ KB
None


In [3]:
# Value to remove
value_to_remove = 'SFL'

# Filter to keep rows where 'col1' is not equal to the value to remove
df = df[df['event_type'] != value_to_remove]

In [4]:
# Get the number of entries for each unique value
value_counts = df['event_type'].value_counts()

# Print the value counts
print(value_counts)

event_type
GRB       2710
SFLARE    1611
TGF       1386
SGR        492
Name: count, dtype: int64


In [5]:
df.head()

,event_name,event_type,reliability
0,bn221109533,SGR,1.0
1,bn220709626,SGR,1.0
2,bn221204313,SGR,1.0
3,bn160727543,SGR,1.0
4,bn220626269,SGR,1.0


In [6]:
# # Shuffle the DataFrame
# df_shuffled = df.sample(frac=1, random_state=42)  # Set random_state for reproducibility

# event_counter = {'GRB': 0, 'SFLARE': 0, 'TGF': 0, 'SGR': 0}
# event_limit = 2000

# # Get a list of all folders in the specified directory
# folders = [folder for folder in os.listdir(data_set_path) if os.path.isdir(os.path.join(data_set_path, folder))]

# folder_names = []
# for folder in folders:
#     if folder.split('_')[0] in event_counter.keys():
#         bn_name = folder.split('_')[0]
#         event_counter[bn_name] += 1
#         folder_names.append(folder.split('_')[1])

# print("events already downloaded\n", event_counter)

# event_counter = {'GRB': 0, 'SFLARE': 0, 'TGF': 0, 'SGR': 0}
# event_type, name = [], []

# for index, row in df_shuffled.iterrows():
#     f = 0
#     for folder in folders:
#         bn_name = folder.split('_')[1]
#         if bn_name == row['event_name']:
#             event_counter[row['event_type']] += 1
#             f = 1
#             break

#     if f == 1:
#         continue

#     if row['event_type'] in event_counter and not (row['event_name'] in folder_names):
#         if event_counter[row['event_type']] < event_limit:
#             event_counter[row['event_type']] += 1
#             event_type.append(row['event_type'])
#             name.append(row['event_name'])

# print('events to download\n',event_counter)



In [7]:
# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=42)  # Set random_state for reproducibility

event_limit = 1100

# Get a list of all folders in the specified directory
folders = [folder for folder in os.listdir(data_set_path) if os.path.isdir(os.path.join(data_set_path, folder))]

# Create sets for faster lookup
folder_set = set(folders)
event_types = set(['GRB', 'SFLARE', 'TGF', 'SGR'])

# Count events already downloaded
downloaded_events = Counter(folder.split('_')[0] for folder in folders if folder.split('_')[0] in event_types)
print("events already downloaded\n", dict(downloaded_events))

# Create set of existing event names for faster lookup
existing_event_names = set(folder.split('_')[1] for folder in folders)

# Initialize counters for events to download
events_to_download = Counter()

# Lists to store new events
new_event_types = []
new_event_names = []

for _, row in df_shuffled.iterrows():
    event_type = row['event_type']
    event_name = row['event_name']

    if event_type in event_types and event_name not in existing_event_names:
        if events_to_download[event_type] < event_limit - downloaded_events[event_type]:
            events_to_download[event_type] += 1
            new_event_types.append(event_type)
            new_event_names.append(event_name)

print('events to download\n', dict(events_to_download))

events already downloaded
 {'GRB': 1655, 'SFLARE': 1085, 'SGR': 492, 'TGF': 1142}
events to download
 {'SFLARE': 15}


In [8]:
print(len(new_event_types))
print(len(new_event_names))

event_list  = new_event_names
event_types = new_event_types

15
15


In [9]:
print(event_list[0])
print(event_types[0])

bn130405735
SFLARE


In [14]:
# reading the ttsplit file
df = pd.read_csv('ttsplit')

# getting the test and train events
test_events = df[df['category'] == 'test']['filename'].tolist()

train_events = df[df['category'] == 'train']['filename'].tolist()

# loading the event names and event types in the list

event_names = []
event_types = []

for _,row in df.iterrows():
    if (row["category"] == "test" or row["category"] == "train") and row["filename"][-1] != 'd':
        event_type,event_name = row['filename'].split("_")
        event_types.append(event_type)
        event_names.append(event_name)

print(len(event_names))
print(len(event_types))


data_set_path = r"D:\GRB_data\new_dataset"

# Get a list of all folders in the specified directory
folders = [folder for folder in os.listdir(data_set_path) if os.path.isdir(os.path.join(data_set_path, folder))]

# Create sets for faster lookup
folder_set = set(folders)
event_types = set(['GRB', 'SFLARE', 'TGF', 'SGR'])

# Count events already downloaded
downloaded_events = Counter(folder.split('_')[0] for folder in folders if folder.split('_')[0] in event_types)
print("events already downloaded\n", dict(downloaded_events))

# Create set of existing event names for faster lookup
existing_event_names = set(folder.split('_')[1] for folder in folders)

# Initialize counters for events to download
events_to_download = Counter()

# Lists to store new events
new_event_types = []
new_event_names = []

for _, row in df.iterrows():
    event_type,event_name = row['filename'].split("_")
    if (row["category"] == "test" or row["category"] == "train") and row["filename"][-1] != 'd':
        if event_type in event_types and event_name not in existing_event_names:
            if events_to_download[event_type] < event_limit - downloaded_events[event_type]:
                events_to_download[event_type] += 1
                new_event_types.append(event_type)
                new_event_names.append(event_name)

print('events to download\n', dict(events_to_download))


event_names = new_event_names
event_types = new_event_types

3492
3492
events already downloaded
 {}
events to download
 {'SFLARE': 1000, 'TGF': 1000, 'SGR': 492, 'GRB': 1000}


In [11]:
print(event_names[0])
print(event_types[0])

bn140204646
SFLARE


In [15]:

dir_path = tools.json_path(r'data_path.json')

# Function to process a single event
def process_event(event_name, event_type,thread_num):
    try:
        temp_thread = r'temp'+ str(thread_num)
        temp_path = os.path.join(dir_path, temp_thread)
        print(temp_path)
        tools.create_folder(temp_path)
        event = event_name
        year = '20' + event[2:4] + "/"

        # URL of the file you want to download : TRIGDAT file
        url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_trigdat_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'
        tools.run_wget_download(url, temp_path)

        # Finding Trigdat file
        trig_string = "_trigdat_"
        trig_pattern = os.path.join(temp_path, 'current', f"*{trig_string}*")
        trigdat_file = glob.glob(trig_pattern)

        # Get the spacecraft pointing from here
        trigdat_filename = trigdat_file[0]

        # URL of the file you want to download : .rsp file
        url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*.rsp*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'
        tools.run_wget_download(url, temp_path)

        # Finding .rsp file
        rsp_string = ".rsp"
        rsp_pattern = os.path.join(temp_path, 'current', f"*{trig_string}*")
        rsp_file = glob.glob(trig_pattern)

        # Getting the RA and DEC
        with fits.open(rsp_file[0], memmap=True) as pha_list:
            ra_obj, dec_obj = (pha_list[0].header['RA_OBJ']), (pha_list[0].header['DEC_OBJ'])

        brightest_nai, bright_nais, brightest_bgo,nai_angles = estimate_source_angles_detectors.angle_to_grb(ra_obj, dec_obj,trigdat_filename)

        for nai_angle,nai in zip(nai_angles,bright_nais):
            if nai == brightest_nai or int(nai_angle) < 60: # only download the brightest detector or if the detector has anlge less than 60 deg
                # URL of the tte file to download
                url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_tte_' + nai + '_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'

                # Construct the wget command
                tools.run_wget_download(url, os.path.join(data_set_path, event_type + '_' + event))
        # Print the size of the queue
        print(event_queue.qsize()) 

    except Exception as e:
        # Get formatted traceback
        traceback_str = traceback.format_exc()

        # Access exception attributes (example)
        if isinstance(e, ZeroDivisionError):
            error_details = "Division by zero attempted."
        else:
            error_details = str(e.args[0]) if len(e.args) else "Unknown error."

        # Custom message based on exception type
        if isinstance(e, ValueError):
            custom_message = "Invalid value provided. Check your input data."
        else:
            custom_message = "An error occurred during processing."

        # Print detailed message
        print(f"""Error:
        - Event: {event_name}
        - Exception: {type(e).__name__}
        - Message: {e}
        - Details: {error_details}
        - Traceback:
        {traceback_str}
        - {custom_message}""")

# Create a queue to store the events
event_queue = Queue()

# Function to run the worker threads
def worker(thread_num):
    print(f"Worker thread {thread_num} started.")

    while True : #
        # Get an event from the queue
        event_name, event_type = event_queue.get()
        print(f"Worker thread {thread_num} processing event {event_type} {event_name}.")

        # Process the event
        process_event(event_name, event_type,thread_num)

        # Signal that the task is completed
        event_queue.task_done()
        print(f"Worker thread {thread_num} processed event {event_name}.")

        if event_queue.qsize() == 0:
            break
    
    print(f"Worker thread {thread_num} finished.")
    
# Create and start the worker threads
num_threads = 2
threads = []
for i in range(num_threads):
    worker_thread = threading.Thread(target=worker, args=(i,), daemon=True)
    worker_thread.start()
    threads.append(worker_thread)

# Add events to the queue
for event_name, event_type in zip(event_names[:2], event_types[:32]):
    event_queue.put((event_name, event_type))

print(event_queue.qsize()) 

# # Wait for all events to be processed
# event_queue.join()
# print("All events have been processed.")

# Wait for all threads to finish
for thread in threads:
    thread.join()

print('done')

Worker thread 0 started.
Worker thread 1 started.
2
Worker thread 0 processing event SFLARE bn140204646.
C:\Users\arpan\OneDrive\Documents\GRB\data\temp0
Worker thread 1 processing event TGF bn190821888.
C:\Users\arpan\OneDrive\Documents\GRB\data\temp1
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp1' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp1' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' created successfully.
download start
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp1' created successfully.
download start
Downloaded bn140204646_trigdat_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp0
download start
Downloaded bn190821888_trigdat_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp1
download start
Downloaded bn1908